In [ ]:
! pip install -q pycocotools

In [ ]:
import shutil

import pandas as pd
from pycocotools.coco import COCO
from sklearn.model_selection import StratifiedGroupKFold
from tqdm.notebook import tqdm

In [ ]:
class CFG:
    n_splits = 4
    fold = 3
    seed = 3000

In [ ]:
coco = COCO("../input/dlsprint2/badlad/labels/coco_format/train/badlad-train-coco.json")
sgkf = StratifiedGroupKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)

In [ ]:
ann_ids = coco.getAnnIds()
anns = coco.loadAnns(ann_ids)

cat_ids = [ann["category_id"] for ann in anns]
img_ids = [ann["image_id"] for ann in anns]

ann_ids = pd.Series(ann_ids)
cat_ids = pd.Series(cat_ids)
img_ids = pd.Series(img_ids)

In [ ]:
indices = [val_idx for _, val_idx in sgkf.split(ann_ids, cat_ids, img_ids)]
fold_idx = indices[CFG.fold]
val_img_ids = set(img_ids[fold_idx])

In [ ]:
cat_ids = coco.getCatIds()
cats = coco.loadCats(cat_ids)
cat_names = [cat["name"] for cat in cats]

for cat_id, cat_name in zip(cat_ids, cat_names):
    ids = coco.getImgIds(imgIds=val_img_ids, catIds=[cat_id])
    print(f"Number of Images Containing {cat_name}: {len(ids)}")

In [ ]:
! mkdir /kaggle/working/images
! mkdir /kaggle/working/labels

In [ ]:
for img in tqdm(coco.loadImgs(val_img_ids)):
    img_src = "/kaggle/input/dlsprint2/badlad/images/train/" + img["file_name"]
    img_dst = "/kaggle/working/images/" + img["file_name"]
    shutil.copy(img_src, img_dst)
    
    label_src = "/kaggle/input/dlsprint2/badlad/labels/yolov8_format/train/" + img["file_name"][:-4] + ".txt"
    label_dst = "/kaggle/working/labels/" + img["file_name"][:-4] + ".txt"
    shutil.copy(label_src, label_dst)